In [ ]:
import os
import code
import shutil

import glob
import h5py
import numpy as np
import tifffile

def stacks_to_hdf5(parent_dir, delete_tiffs):
    """
    Convert tiff/ome.tiff files to 

    TODO:
    - Check that loading the first ome.tiff file with this method only loads
        one frame. Remember that sometimes loading the first of an ome.tiff
        series for some software causes it to load the whole series.
        Flag for suppressing this behavior.
    - 'fnames' and 'f_out' should have platform-agnostic path concatenation
        rather than escaped slashes ("//"). 
        
        e.g. import os
        os.path.join(os.path.curdir, 'file.name')
    """
    
    fnames = sorted(glob.glob(parent_dir + "//.tif"))
    first_fname_handle = tifffile.TiffFile(fnames[0])

    stack_size = len(first_fname_handle.pages)
    img_width, img_height = first_fname_handle.pages[0]
    
    if stack_size == 1: # Single frames per file in old acquisitions
        first_30 = np.zeros((30, img_width, img_height))
        last_30  = np.zeros((30, img_width, img_height))
        last_30_fnames = fnames[-30:]
        for i in range(30):
            first_30[i,:,:] = tifffile.imread(fnames[i], is_ome=False))
            last_30[i,:,:]  = tifffile.imread(last_30_fnames, is_ome=False)
        num_frames = len(fnames) + 60
        
    else: # Stacks of frames per file.
        # All except last stack ought to be same size.
        for i in range(1, len(fnames - 1)):
            tif_handle = tifffile.TiffFile(fnames[i])
            num_pages_in_tif = len(tif_handle.pages)
            assert num_pages_in_tif == stack_size, \
                "Stack sizes inconsistent : first stack is of size {}, \
                but file {} has {} pages. Data may be malformed. \
                Investigate {}".format(stack_size, fname[i], num_pages_in_tif, parent_dir)
        mov = tifffile.imread(fnames[i])
        first_30 = mov[0:30,:,:]
        last_stack = tifffile.imread(fnames[-1])
        last_30 = last_stack[-30:,:,:]
        num_frames = (stack_size * (len(fnames) - 1)) + \
                        last_stack.shape[0] + 60
    
    f_out = h5py.File(parent_dir + "//unregistered.h5", "w")
    f_out.create_dataset("mov", (num_frames, img_width, img_height))

    # First and last 30 frames removed by DeepInterpolation moving window
    f_out["mov"][0:30,:,:] = np.flip(first_30, axis=0)
    
    # Adding the acquisition itself in the middle
    for i in range(len(fnames) - 1):
        index = 30 + (i * stack_size)
        mov = tifffile.imread(fnames[i], is_ome=False)
        f_out["mov"][index:index + stack_size,:,:] = mov
        
    # Add last 30 flipped, updating index with size of last loaded frame file.
    index = index + mov.shape[0] if stack_size == 1 else index + 1
    f_out["mov"][index:,:,:] = np.flip(last_30, axis=0)
    f_out.close() # Done.
                      
def deinterleave_movies(parent_dir, scope_format):
    """
    Splits tiff stacks where two channels have been interleaved.
    Matches how our SCANIMAGE acquisitions are done for 2ch.

    TODO : 
        - escaped slashes aren't system-agnostic. Change this w/ os.path?
    """
    is_bruker = 'BRUKER' in parent_dir
    is_scanimage = 'SCANIMAGE' in parent_dir

    fnames = glob.glob(parent_dir + '//*.tif')
    assert len(fnames) > 0, "No valid tif files found in {}".format(parent_dir)
    
    os.mkdir(parent_dir + '//ch1')
    os.mkdir(parent_dir + '//ch2')

    if scope_format == "BRUKER":
        # Separate files for each
        ch_1_fnames = [f for f in fnames if 'Ch1' in f]
        ch_2_fnames = [f for f in fnames if 'Ch2' in f]
        # Not presuming same number for both, but there usually should be.
        for i in range(len(ch_1_fnames)):
            shutil.move(ch_1_fnames[i], "//".join([parent_dir, 'ch1', f"{i:05d}", ".tif"]))
        for i in range(len(ch_2_fnames)):
            shutil.move(ch_2_fnames[i], "//".join([parent_dir, 'ch2', f"{i:05d}", ".tif"]))
            
    elif scope_format == "SCANIMAGE":
        # Interleaved frames in the same file. Read, split, write separate channels.
        for i in range(len(fnames)):
            mov = tifffile.imread(fnames[i])
            ch_1 = mov[::2,:,:] # Even frames, including first
            ch_2 = mov[1::2,:,:]# Odd frames
            tifffile.imwrite('//'.join([parent_dir, 'ch1', f"{i:05d}", "*.tif"]), ch_1)
            tifffile.imwrite('//'.join([parent_dir, 'ch2', f"{i:05d}", "*.tif"]), ch_2)
            
    else:
        assert False, "{} is improper scope format. \
                Give either BRUKER or SCANIMAGE as second argument.".format(scope_format)
        
        




